In [ ]:
# HTML output for this notebook can be produced using
# jupyter nbconvert --to html --no-input notebook_name.ipynb
# or
# jupyter nbconvert --to pdf --no-input notebook_name.ipynb
# edit title and authors in notebook metadata (e.g. jupyter lab / notebook tools / advanced)

See [TODO.md](../TODO.md) for outstanding items.

This notebook contains the same functionality as `tidal_analysis_rt.ipynb`, but with all detail code extracted into the `../acoustic_tracking` package code folder.  For this notebook to run properly, ensure:
* `acoustic_tracking` is downloaded and installed on your local machine according to the installation instructions
* The `acoustic_env` conda environment has been activated

Analysis of acoustic tracking performance using range test data obtained in the Northern Straight of Georgia near Comox, BC, between January 2018 and April 2019. 

Components of this notebook:

 * load detection events and deployment metadata about receivers and transmitters
 * analyse detections for each receiver/transmitter combination separately
 * calculate interval lengths between detections and detection rates within fixed time windows
 * calculate further metadata descriptions, e.g. distance between stations, R/T names involving distinctions such as near/far distance or high/low power
 * obtain further environmental variables from external data sources
 * produce different visualizations to study correlation of env. variables and operating conditions with detection rate

**Data:** For most sources for environmental variables data has been collected for 3 hour intervals.

In [ ]:
# Import packages
# from acoustic_tracking import *
import acoustic_tracking as at
import copy
import pandas as pd
import kadlu
from datetime import datetime
import numpy as np
import sys
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
from IPython.display import display, Markdown
from matplotlib.pyplot import rcParams

# Setup matplotlib inline plotting
%matplotlib inline
at.mpl_set_notebook_params()

show_details = False

In [ ]:
#at.reload_acoustic()

In [ ]:
baseconfig = at.yload(at.load_file("../base_config.yaml"))
viewconfig = at.yload(at.load_file("../view_config.yaml"))
at.deep_update(baseconfig, viewconfig)

rawconfig = at.yload(at.load_file("nsog_config.yaml"))
config = at.merge_dicts(baseconfig, rawconfig)
at.prepare_config(config)

In [ ]:
#print(at.ydump(config))

In [ ]:
import kadlu
from datetime import datetime

import pandas as pd
import numpy as np
from IPython.display import Markdown, display
import matplotlib.pyplot as plt
import seaborn as sns

# mpl.use('module://ipympl.backend_nbagg')
#%matplotlib widget
%matplotlib inline
at.mpl_set_notebook_params()

show_details = config.settings.show_details

## Read Raw Detection Data & Metadata

In [ ]:
# at.reload_acoustic()

In [ ]:
detection_df, mdb = at.read_via_config(config)

In [ ]:
if config.settings.otn_transmitter_patch:
    at.otn_transmitter_patch_1(mdb)

In [ ]:
df_dets, df_inits, _ = at.process_intervals(detection_df, mdb)

In [ ]:
detection_df, event_bin_split = at.detection_rate_grid(df_dets, config.settings.time_bin_length, mdb)
events_df, bins_df = at.split_by_index(detection_df, event_bin_split)

# Group detection data by (Receiver, Transmitter) pairs

A name is produced for each pairing that reflects their configuration, such as power level, tag family, distance - as determined by parsing the metadata.

In [ ]:
deploy_lat_lon = mdb.deploy.groupby('STATION_NO')[['DEPLOY_LAT','DEPLOY_LONG']].nth(0)
mdb.station_dists_m = at.calc_station_dists_m(deploy_lat_lon)

if "Receiver/Transmitter" not in mdb.rt_groups.columns:
    at.add_rt_group_info(events_df, mdb)

In [ ]:
if config.settings.show_details:
    at.report_station_info(mdb)

In [ ]:
if config.view.show_dr_plots:
    at.displaymd("# Detection rate plots for data screening")
    for gn, tgroup in bins_df.reset_index().groupby(['Receiver','Transmitter']):
        at.plot_group_dr(gn, tgroup, mdb)
        plt.show()

## Detection data is merged with environmental variables from kadlu

[Kadlu](https://docs.meridian.cs.dal.ca/kadlu/index.html#) is a Python package which provides functionality for fetching and interpolating environmental data related to ocean ambient nose levels. The `acoustic_tracking` package provides users with the option to integrate environmental data from Kadlu with their own detection datasets. 


To extract environmental data from kadlu, you will need to specify (1) data sources and (2) bounds. Then, using these specifications you can use the `add_kadlu_env_data()` function to automatically extract and interpolate data using the kadlu Python package. 

### Data Sources
A `sources` dictionary is used to specify the variables and data sources you want to retrieve using kadlu. To see a high level overview of available data sources, execute the command below. 
```python
print(kadlu.source_map)
``` 

In [ ]:
print(kadlu.source_map)

In [ ]:
# List Sources
sources = config.data.sources

### Data Boundaries
A `bounds` dictionary is used to specify the spatial and temporal boundaries for which you want to retrieve data. A `north`, `south`, `east`, and `west` value are provided to specify geospatial boundaries, while a `start` and `end` are used to specify temporal boundaries. Optionally, `top` and `bottom` values can be used to limit data to specific depths. 

In [ ]:
# Define Bounds
# Mahone Bay, NS

lat_center = 44.5541333
north_offset = 0.5
south_offset = 2

lon_center = -64.17682
east_offset = 2
west_offset = 2

bounds = dict(start=datetime(2016, 3, 9), end=datetime(2016,3,11),
              south=lat_center - south_offset, west=lon_center - west_offset, 
              north=lat_center + north_offset, east= lon_center + east_offset, 
              top=0, bottom=0)

# OR

bounds = config.bounds

print(bounds)

Sometimes, it can be helpful to have a map visualization of the boundaries you are setting. Here, you can use the `plot_bounds` function to see the boundaries you have specified. Optionally, you can provide receiver locations and metadata to the `plot_bounds` function as well. 

In [ ]:
# Retrieve receiver information
receiver_info = detection_df[['Receiver.lat', 'Receiver.lon', 'Receiver.ID', 
                              'Receiver', 'Receiver.depth']].drop_duplicates()
receiver_locations_df = detection_df[['Receiver.lat', 'Receiver.lon']].drop_duplicates()
receiver_locations = list(zip(receiver_locations_df['Receiver.lat'], receiver_locations_df['Receiver.lon']))

try:
    node_locs = set(list(zip(kadlu_result[1], kadlu_result[2])))
except:
    node_locs = []
    print("Warning: no node locations available. Consider to run this cell again, after the next cell set result approprietly.")

at.plot_bounds(bounds, receiver_locations=receiver_locations, 
               receiver_info=receiver_info, node_locations=node_locs);

### Add Environmental Variables from kadlu


In [ ]:
df_detections_env, kadlu_result = at.add_kadlu_env_data(bounds, sources, detection_df)

### Add further calculated columns and prepare for event/bin row split

In [ ]:
event_bin_dfs = lambda df: at.split_by_index(df, event_bin_split)
detection_events_df, detection_bins_df = event_bin_dfs(df_detections_env)
rt_group_data = list(df_detections_env.groupby(['Receiver','Transmitter']))

In [ ]:
columns, colnames, column = at.get_column_info(config)

at.displaymd("# Plots of detection density and interval lengths <br/> against tidal phase (t2) and {}".format(colnames[column]))

In [ ]:
from acoustic_tracking.plotting import heatmaps

#det_df = detection_events_df
det_df = detection_bins_df

features = det_df[['wavedir', 'waveheight', 'waveperiod','wind_uv', 'wind_u', 'wind_v', 'detection_rate']]

heatmaps.plot_feature_heatmap(features, method='spearman');